In [1]:
import pandas as pd 
import os, time, json
from tqdm.notebook import tqdm_notebook
with open('/Users/carlo/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [2]:
import tmdbsimple as tmdb
tmdb.API_KEY = login['api-key']

In [3]:
FOLDER = "Data/"
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'final_tmdb_data.csv.gz',
 'movies-ERD.png',
 'title-akas-us-only.csv',
 'title.basics.csv',
 'title.ratings.csv',
 'tmdb_api2001.json',
 'tmdb_api2002.json',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json',
 'tmdb_api_results_2002.json',
 'tmdb_data_2000.csv.gz',
 'tmdb_data_2001.csv.gz',
 'tmdb_data_2002.csv.gz']

In [4]:
basics = pd.read_csv('Data/title.basics.csv')
basics.info()
basics.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86979 entries, 0 to 86978
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      86979 non-null  int64  
 1   tconst          86979 non-null  object 
 2   titleType       86979 non-null  object 
 3   primaryTitle    86979 non-null  object 
 4   originalTitle   86979 non-null  object 
 5   isAdult         86979 non-null  int64  
 6   startYear       86979 non-null  float64
 7   endYear         0 non-null      float64
 8   runtimeMinutes  86979 non-null  int64  
 9   genres          86979 non-null  object 
dtypes: float64(2), int64(3), object(5)
memory usage: 6.6+ MB


,Unnamed: 0,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama


In [5]:
def get_movie_with_rating(movie_id):
    """Adapted from source = https://github.com/celiao/tmdbsimple"""
    
    movie = tmdb.Movies(movie_id)
# save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
# Loop through countries in releases
    for c in releases['countries']:
    # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
        ## save a "certification" key in the info dict with the certification
           info['certification'] = c['certification']
    return info

In [6]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [7]:
test_ids = ['tt0332280','tt0848228']
results = []
for movie_id in test_ids:
    try:
        movie_info = get_movie_with_rating(movie_id)
        results.append(movie_info)
    except: 
        pass
pd.DataFrame(results)

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,False,/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg,None,29000000,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",http://www.newline.com/properties/notebookthe....,11036,tt0332280,en,The Notebook,...,115600000,123,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Behind every great love is a great story.,The Notebook,False,7.886,10874,PG-13
1,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,"{'id': 86311, 'name': 'The Avengers Collection...",220000000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.marvel.com/movies/the-avengers,24428,tt0848228,en,The Avengers,...,1518815515,143,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Some assembly required.,The Avengers,False,7.712,29617,PG-13


In [8]:
YEARS_TO_GET = [2001,2002]

In [9]:
# Start of OUTER loop
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f"{FOLDER}tmdb_api{YEAR}.json"
    file_exists = os.path.isfile(JSON_FILE)
# If it does not exist: create it
    if file_exists == False:
    # Print a message indicating the file is being created 
        print(f"Creating {JSON_FILE} for API results for {YEAR}.")
    # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    else:
        print('The file {JSON_FILE} already exists.')
#Saving new year as the current df
    df = basics.loc[ basics['startYear'] == YEAR].copy()
# saving movie ids to separate variable
    movie_ids = df['tconst']
# Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
# filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]


YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

The file {JSON_FILE} already exists.
The file {JSON_FILE} already exists.


In [10]:
YEAR = 2000
errors = []

In [11]:
# Loop through movie_ids_to_get with a tqdm progress bar
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):
    try:
    # Retrieve then data for the movie id
        temp = get_movie_with_rating(movie_id)  
    # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
    # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
    
    except Exception as e:
        errors.append([movie_id, e])

Movies from 2000:   0%|          | 0/276 [00:00<?, ?it/s]

In [12]:
print(f"- Totall errors: {len(errors)}")

- Totall errors: 276


In [13]:
df_2000 = pd.read_json(JSON_FILE)
df_2000.to_csv(f"{FOLDER}tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

In [14]:
YEAR = 2001
error = []

In [15]:
# Loop through movie_ids_to_get with a tqdm progress bar
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):
    try:
    # Retrieve then data for the movie id
        temp = get_movie_with_rating(movie_id)  
    # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
    # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
    
    except Exception as e:
        errors.append([movie_id, e])

Movies from 2001:   0%|          | 0/276 [00:00<?, ?it/s]

In [16]:
print(f"- Totall errors: {len(errors)}")

- Totall errors: 552


In [17]:
df_2001 = pd.read_json(JSON_FILE)
df_2001.to_csv(f"{FOLDER}tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

In [18]:
YEAR = 2002
error = []

In [19]:
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):
    try:
    # Retrieve then data for the movie id
        temp = get_movie_with_rating(movie_id)  
    # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
    # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
    
    except Exception as e:
        errors.append([movie_id, e])

Movies from 2002:   0%|          | 0/276 [00:00<?, ?it/s]

In [20]:
print(f"- Totall errors: {len(errors)}")

- Totall errors: 828


In [21]:
df_2002 = pd.read_json(JSON_FILE)
df_2002.to_csv(f"{FOLDER}tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

In [22]:
import glob
# Use glob to get all filepaths that match the pattern (*=wildcard)
tmdb_files = sorted(glob.glob("Data/tmdb_data*.csv.gz"))
tmdb_files

['Data\\tmdb_data_2000.csv.gz',
 'Data\\tmdb_data_2001.csv.gz',
 'Data\\tmdb_data_2002.csv.gz']

In [35]:
# Use read_csv in a list comprehension and combine with concat to load all files
df = pd.concat([pd.read_csv(f) for f in tmdb_files] )
df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0096056,0.0,/95U3MUDXu4xSCmVLtWgargRipDi.jpg,NaN,0.0,"[{'id': 18, 'name': 'Drama'}]",NaN,109809.0,en,Crime and Punishment,...,0.0,126.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Crime and Punishment,0.0,4.900,16.0,NaN
2,tt0118926,0.0,/qR3Dk3ctnrrxkAI6I472RhamIbu.jpg,NaN,0.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,20689.0,en,The Dancer Upstairs,...,5227348.0,132.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"An honest man caught in a world of intrigue, p...",The Dancer Upstairs,0.0,6.093,54.0,NaN
3,tt0119980,0.0,NaN,NaN,0.0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,563364.0,en,Random Shooting in LA,...,0.0,91.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,NaN,Random Shooting in LA,0.0,0.000,0.0,NaN
4,tt0120679,0.0,/s04Ds4xbJU7DzeGVyamccH4LoxF.jpg,NaN,12000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...",https://www.miramax.com/movie/frida,1360.0,en,Frida,...,56298474.0,123.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,Prepare to be seduced.,Frida,0.0,7.421,1928.0,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3891,tt6449044,0.0,/a9pkw8stijESGx1flSGPqcXLkHu.jpg,"{'id': 957260, 'name': 'The Conman Collection'...",0.0,"[{'id': 35, 'name': 'Comedy'}]",NaN,314105.0,cn,賭俠2002,...,0.0,97.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,NaN,The Conman 2002,0.0,4.300,3.0,NaN
3892,tt6694126,0.0,/sXjVpTZyDvwzPVZve3AmyCUBeHk.jpg,NaN,0.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,819174.0,fa,عروس خوش‌قدم,...,0.0,101.0,"[{'english_name': 'Persian', 'iso_639_1': 'fa'...",Released,NaN,The Lucky Bride,0.0,8.000,1.0,NaN
3893,tt8302928,0.0,NaN,NaN,0.0,"[{'id': 16, 'name': 'Animation'}, {'id': 12, '...",NaN,866533.0,el,Movie Toons: Treasure Island,...,0.0,0.0,[],Released,NaN,Movie Toons: Treasure Island,0.0,0.000,0.0,NaN
3894,tt8474326,0.0,NaN,NaN,0.0,[],NaN,292027.0,en,Skin Eating Jungle Vampires,...,0.0,0.0,[],Released,NaN,Skin Eating Jungle Vampires,0.0,0.000,0.0,NaN
